In [14]:
import websocket
import json 
import logging
from datetime import datetime
import csv


class CryptoExtractSchedule:
    def __init__(self,streams,interval,output_path):
        self.streams=streams
        self.interval=interval
        self.output_path= output_path
        self.json_data_collection=[]
    
    def write_to_csv(self):
        print(f'Data collection Value = {self.json_data_collection}')###

        if((not self.json_data_collection) and len(self.json_data_collection)==0):
            return

        dt_string = datetime.now().strftime("%d-%m-%Y %H:%M:%S").replace(" ", "").replace(':','-')
        final_output_path= self.output_path+'cryptoData'+dt_string+'.csv'
        data_file = open(final_output_path, 'w', newline='')
        csv_writer = csv.writer(data_file)
        count = 0
        for data in self.json_data_collection:
            if count == 0:
                header = data.keys()
                csv_writer.writerow(header)
                count += 1
            csv_writer.writerow(data.values()) 
        data_file.close()
        #reset value
        self.json_data_collection=[]
    
    def on_open(self,ws):
        print("Binance connected ...")
        self.prev_date=datetime.now()
        try:
            subscribe = {"method":"SUBSCRIBE","params":self.streams,"id":1}
            ws.send(json.dumps(subscribe))
        except Exception as e:
            logging.ERROR(e)
            print(e)

    def on_message(self,ws, message):
        json_message = json.loads(message)
        is_candle_closed=False
        if(json_message and  'k' in json_message):
            candle = json_message['k']
            is_candle_closed= candle['x']
        if(is_candle_closed):
            print(f'inisde is candle closed with candle value = {candle}' )### inside isCandle closed
            self.json_data_collection.append(candle)
        
        #custom sceduling, inserting data to csv every 10 mins
        self.cur_date = datetime.now()
        if(((self.cur_date - self.prev_date).seconds)/60 >=10):
            self.write_to_csv()
            self.prev_date=self.cur_date

        # print(self.json_data_collection)
        #after every 10 minutes dump the data to a csv file
        #schedule.every(1).minutes.do(self.write_to_csv)
        

    def on_error(self,ws, error):
        ###logging.ERROR(error)
        print(error)

    def on_close(self,ws):
        print("Connection Closed!")

    # def get_socket(self):
    #     socket = 'wss://stream.binance.com:9443/ws'
    #     self.ws = websocket.WebSocketApp(socket,on_open= lambda ws,msg: self.on_open(ws),
    #                                      on_close= lambda ws: self.on_close(ws), 
    #                                      on_message = lambda ws,msg: self.on_message(ws,msg), 
    #                                      on_error= lambda ws,msg: self.on_error(ws,msg))
    #     return self.ws

    def run(self):
        socket = 'wss://data-stream.binance.com/ws'
        self.ws = websocket.WebSocketApp(socket,on_open= self.on_open,
                                         on_close= self.on_close, 
                                         on_message = self.on_message, 
                                         on_error= self.on_error)
        self.ws.run_forever()


interval = '1m'
streams = [f"ethusdt@kline_{interval}",f"btcusdt@kline_{interval}",f"xrpusdt@kline_{interval}",f"bnbusdt@kline_{interval}",f"dogeusdt@kline_{interval}",f"adausdt@kline_{interval}",f"maticusdt@kline_{interval}",
f"dotusdt@kline_{interval}",f"solusdt@kline_{interval}",f"usdcusdt@kline_{interval}"]

#TODO : FIX logging
logging.basicConfig(filename='GetCryptoData.log', level=logging.ERROR, format='%(asctime)s:%(levelname)s:%(message)s')

file_path = '/Users/pratyushpradhan/Developer/Personal/Projects/Crypto/RealTimeData/'
obj = CryptoExtractSchedule(streams,interval,file_path)
temp = obj.run()

Binance connected ...
inisde is candle closed with candle value = {'t': 1670317680000, 'T': 1670317739999, 's': 'XRPUSDT', 'i': '1m', 'f': 492314193, 'L': 492314347, 'o': '0.38230000', 'c': '0.38250000', 'h': '0.38250000', 'l': '0.38220000', 'v': '485676.00000000', 'n': 155, 'x': True, 'q': '185704.52990000', 'V': '278400.00000000', 'Q': '106457.02060000', 'B': '0'}
inisde is candle closed with candle value = {'t': 1670317680000, 'T': 1670317739999, 's': 'BTCUSDT', 'i': '1m', 'f': 2294102665, 'L': 2294106007, 'o': '17024.74000000', 'c': '17033.42000000', 'h': '17034.00000000', 'l': '17024.49000000', 'v': '153.16007000', 'n': 3343, 'x': True, 'q': '2608157.38924070', 'V': '86.17482000', 'Q': '1467495.33471150', 'B': '0'}
inisde is candle closed with candle value = {'t': 1670317680000, 'T': 1670317739999, 's': 'ETHUSDT', 'i': '1m', 'f': 1037724540, 'L': 1037724824, 'o': '1261.43000000', 'c': '1262.39000000', 'h': '1262.40000000', 'l': '1261.42000000', 'v': '137.23640000', 'n': 285, 'x': 

In [4]:

SOCKET = "wss://data-stream.binance.com/ws/btcusdt@kline_1m"

def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('close connection')

def on_message(ws, message):
    print('received message')
    print(message)

def on_error(ws, message):
    print('error:', message)


ws = websocket.WebSocketApp(SOCKET, on_open = on_open, on_close = on_close, on_message = on_message, on_error = on_error)
ws.run_forever()

opened connection
received message
{"e":"kline","E":1670315170139,"s":"BTCUSDT","k":{"t":1670315160000,"T":1670315219999,"s":"BTCUSDT","i":"1m","f":2293957482,"L":2293958127,"o":"17012.95000000","c":"17012.28000000","h":"17014.42000000","l":"17011.35000000","v":"28.39120000","n":646,"x":false,"q":"483025.49075620","V":"15.28915000","Q":"260121.92390090","B":"0"}}
received message
{"e":"kline","E":1670315172228,"s":"BTCUSDT","k":{"t":1670315160000,"T":1670315219999,"s":"BTCUSDT","i":"1m","f":2293957482,"L":2293958201,"o":"17012.95000000","c":"17012.03000000","h":"17014.42000000","l":"17011.35000000","v":"31.07375000","n":720,"x":false,"q":"528661.02134140","V":"15.38486000","Q":"261750.18225620","B":"0"}}
error: 
error: on_close() takes 1 positional argument but 3 were given


True

In [11]:
def printHell():
    return
    print(hello)
printHell()